# Hubble Ultra Deep Field Final Project

### Import numpy and sep

In [ ]:
import numpy as np
import sep

### Import matplotlib tools and fits, set figure size

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams
import scipy.ndimage as ndimage

%matplotlib inline


rcParams['figure.figsize'] = [10.,8.]


### Open image file

In [ ]:
# read image into standard 2-d numpy array
fname = "hudf.fits"
hdu_list = fits.open(fname)
hdu_list.info()

### Get image data

In [ ]:
image_data = fits.getdata(fname)
print(type(image_data))
print(image_data.shape)

### Adjust image data to get positive values, correct image orientation

In [ ]:
data = image_data.byteswap().newbyteorder() #fixes negative values
data = ndimage.rotate(image_data, 40, reshape=True) #adjusts orientation of image

### Display the image in grayscale

In [ ]:
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.xlim([1500,3600]) #adjust x axis
plt.ylim([1500,3600]) #adjust y axis 
plt.savefig('hduf.png')#save image

### Read the background data

In [ ]:
bkg = sep.Background(data)

### Get background mean and noise

In [ ]:
print(bkg.globalback) #background mean
print(bkg.globalrms) #background noise

### Set background as array

In [ ]:
bkg_image = bkg.back()

### Show background image in grayscale

In [ ]:
plt.imshow(bkg_image, interpolation = 'nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.xlim([1500,3600])#adjust x axis
plt.ylim([1500,3600])#adjust y axis
plt.savefig('hudf_background.png') #save image

### Sets background noise as array

In [ ]:
bkg_rms = bkg.rms()

### Show background noise image

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest',cmap='gray',origin='lower')
plt.colorbar();
plt.xlim([1500,3600])#adjust x axis
plt.ylim([1500,3600])#adjust y axis
plt.savefig('hudf_bkg_rms.png') #save image

### Subtract the background from original data

In [ ]:
data_sub = data - bkg

### Run object detection

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

### Number of objects detected

In [ ]:
len(objects)

### 8725 objects detected 

### Plot image with objects marked

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m,s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180./np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    #plt.savefig('objectdetection_image1.png')
plt.xlim([1500,3600])#adjust x axis
plt.ylim([1500,3600])#adjust y axis

### Plot histogram in AB magnitude

In [ ]:
data_hist = -2.5*np.log(data) - 21.1 #converts flux to AB magnitude
histogram = plt.hist(data_hist.flatten(),bins='auto', range=(-30,30)) #plots histogram
